## SMDA Project- Jeremy Dorsey
1. Introduction
For this project, I collected statistics from this year’s Major League Baseball (MLB) season.
I gathered many variables including wins (Wins); runs scored (RS); on-base percentage (OBP); earned run average (ERA); walks and hits per inning pitched (WHIP); fielding percentage (FPCT); defensive runs saved (DRS); weighted on-base average (wOBA); and fielding independent pitching (FIP). 
After this analysis, I hope to be able to see not only which variables predict wins, but also which kind of statistics predict wins better.
I would also like to answer the following questions: 
After finding which statistics predict wins best, which other statistics contribute highly to those predictors? What should teams look for in some players when evaluating or putting a team together?

In [1]:
# This R environment comes with all of CRAN preinstalled, as well as many other helpful packages
# The environment is defined by the kaggle/rstats docker image: https://github.com/kaggle/docker-rstats
# For example, here's several helpful packages to load in 

library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

system("ls ../input/SMDAProjectdata.txt")
MLBData <- read_csv("../input/SMDAProjectdata.txt")


# Any results you write to the current directory are saved as output.

NameError: name 'library' is not defined

In [2]:
data <- read.table("../input/SMDAProjectdata.txt", sep = "", header = T)


NameError: name 'data' is not defined

In [3]:
head(data)

In [4]:
data$RS

In [6]:
#Scatterplots
plot(data$RS,data$WINS)
plot(data$OBP,data$WINS)
plot(data$ERA,data$WINS)
plot(data$WHIP,data$WINS)
plot(data$FPCT,data$WINS)
plot(data$DRS,data$WINS)
plot(data$wOBA,data$WINS)
plot(data$FIP,data$WINS)


### Discussion
Many of the variables had a decent linear with wins. ERA looks to have the highest negative correlation with wins and On-base Percentage looks to have the highest positive correlation.

In [8]:
##Correlation matrix
cor(cbind(data$RS,data$OBP,data$ERA,data$WHIP,data$FPCT,data$DRS,data$wOBA,data$FIP))

In [9]:
mod.RS<-lm(data$RS~data$OBP+data$ERA+data$WHIP+data$FPCT+data$DRS+data$wOBA+data$FIP)
summary(mod.RS)
#VIF.RS<-1/(1-.8874)
#VIF.RS

In [10]:
mod.wOBA<-lm(data$wOBA~data$RS+data$OBP+data$ERA+data$WHIP+data$FPCT+data$DRS+data$FIP)
summary(mod.wOBA)
#VIF.wOBA<-1/(1-.9069)
#VIF.wOBA
mod.OBP<-lm(data$OBP~data$RS+data$ERA+data$WHIP+data$FPCT+data$DRS+data$wOBA+data$FIP)
summary(mod.OBP)
#VIF.OBP<-1/(1-.7909)
#VIF.OBP
mod.ERA<-lm(data$ERA~data$RS+data$OBP+data$WHIP+data$FPCT+data$DRS+data$wOBA+data$FIP)
summary(mod.ERA)
#VIF.ERA<-1/(1-.8778)
#VIF.ERA
mod.WHIP<-lm(data$WHIP~data$RS+data$OBP+data$ERA+data$FPCT+data$DRS+data$wOBA+data$FIP)
summary(mod.WHIP)
#VIF.WHIP<-1/(1-.8737)
#VIF.WHIP
mod.FIP<-lm(data$FIP~data$RS+data$OBP+data$ERA+data$WHIP+data$FPCT+data$DRS+data$wOBA)
summary(mod.FIP)
#VIF.FIP<-1/(1-.687)
#VIF.FIP


In [11]:
##full model for R^2
mod1<-lm(data$WINS~data$RS+data$OBP+data$ERA+data$WHIP+data$FPCT+data$DRS+data$wOBA+data$FIP)
summary(mod1)

## Next steps in discussion
A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis. A large p-value (> 0.05) indicates weak evidence against the null hypothesis, so you fail to reject the null hypothesis. We need to figure out whether or not our response variable (WINS) has any relation with our other eight predictor variables.
![P-value explainer](http://imgs.xkcd.com/comics/p_values.png)

In [13]:
##F-test
pf(15.59,8,21,lower.tail=F)

In [14]:
library(MASS)

In [15]:
optimal1<-stepAIC(mod1)


In [17]:
mod.2<-lm(data$WINS~data$FPCT+data$RS+data$ERA)
summary(mod.2)

In [29]:
X.scale<-scale(cbind(data$RS,data$OBP,data$ERA,data$WHIP,data$FPCT,data$DRS,data$wOBA,data$FIP))  
##to find the influential observations
cooks.distance(mod1)
qf(.50,8,21)

![Outlier description](http://www2.southeastern.edu/Academics/Faculty/dgurney/Outlier.jpg)

In [30]:
baseball0<-data
datanew<-cbind(baseball0,data$RS,data$ERA,data$FPCT)
X<-as.matrix(cbind(datanew$RS,datanew$ERA,datanew$FPCT))
Y<-datanew$WINS
set.seed(5)
n<-dim(datanew)[1]
Group1.index<-sample(1:n,round(n/2),replace=F)
Group2.index<-setdiff(1:n,Group1.index) 
Group1.X <- X[Group1.index,]; Group1.Y <- Y[Group1.index]
Group2.X <- X[Group2.index,]; Group2.Y <- Y[Group2.index]
mod.Group1<-lm(Group1.Y~Group1.X)
mod.Group2<-lm(Group2.Y~Group2.X)
fitted.Group2 <- cbind(rep(1,length(Group2.index)),Group2.X)%*%mod.Group1$coefficients
fitted.Group1 <- cbind(rep(1,length(Group1.index)),Group1.X)%*%mod.Group2$coefficients
rsquared.Group2 <- 1 - sum((Group2.Y-fitted.Group2)^2)/sum((Group2.Y-mean(Group2.Y))^2)
rsquared.Group1 <- 1 - sum((Group1.Y-fitted.Group1)^2)/sum((Group1.Y-mean(Group1.Y))^2)
mean(c(rsquared.Group2,rsquared.Group1))
summary(mod.2)

In [33]:
baseball0<-data
datanew<-cbind(baseball0,data$RS,data$ERA)
X<-as.matrix(cbind(datanew$RS,datanew$ERA))
Y<-datanew$WINS
set.seed(5)
n<-dim(datanew)[1]
Group1.index<-sample(1:n,round(n/2),replace=F)
Group2.index<-setdiff(1:n,Group1.index) 
Group1.X <- X[Group1.index,]; Group1.Y <- Y[Group1.index]
Group2.X <- X[Group2.index,]; Group2.Y <- Y[Group2.index]
mod.Group1<-lm(Group1.Y~Group1.X)
mod.Group2<-lm(Group2.Y~Group2.X)
fitted.Group2 <- cbind(rep(1,length(Group2.index)),Group2.X)%*%mod.Group1$coefficients
fitted.Group1 <- cbind(rep(1,length(Group1.index)),Group1.X)%*%mod.Group2$coefficients
rsquared.Group2 <- 1 - sum((Group2.Y-fitted.Group2)^2)/sum((Group2.Y-mean(Group2.Y))^2)
rsquared.Group1 <- 1 - sum((Group1.Y-fitted.Group1)^2)/sum((Group1.Y-mean(Group1.Y))^2)
mean(c(rsquared.Group2,rsquared.Group1))
mod.3<-lm(data$WINS~data$RS+data$ERA)
summary(mod.3)

In [35]:
mod.3<-lm(data$WINS~data$RS+data$ERA)
summary(mod.3)
CritVal<-qt(.025,27)
c(.07575-CritVal*.01519,.07575+CritVal*.01519)

In [36]:
c(-17.03099-CritVal*1.99153,-17.03099+CritVal*1.99153)

In [49]:
mean(data$RS)
mean(data$ERA)


In [47]:
predict(mod.3,data.frame(data$RS=mean(data$RS),data$ERA=mean(data$ERA)),interval="prediction",level=0.95)

## Explanation
Does not want to read the data as a mean. But the average number of wins is 81. And this code should produce a prediction interval of (70.83079, 90.96921).